<a href="https://colab.research.google.com/github/ZoraizQ/brain_mas_classifier/blob/master/brain_mas_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifier for Human Brain Mental Attention State using EEG BCI-based data

In [69]:
import os
import random
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from scipy.io import loadmat
import pandas as pd

import tensorflow as tf
from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Activation, BatchNormalization, LeakyReLU, Dropout
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.utils import plot_model


SAMP_FREQ = 128 # SAMPLING FREQUENCY Hz (samples/second)

In [11]:
! pip install --upgrade --force-reinstall --no-deps kaggle

from google.colab import files
files.upload() #upload kaggle.json

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Processing /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303/kaggle-1.5.12-cp37-none-any.whl
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [17]:
! [ ! -d /content/data ] && kaggle datasets download -d 'inancigdem/eeg-data-for-mental-attention-state-detection' -p '/content/data' --unzip

# Data Preprocessing

In [48]:
DATA_FOLDER = '/content/data'

filenames = np.array(glob(os.path.join(DATA_FOLDER, '*/*.mat')))

NUM_FILES_ALL = len(filenames) # with habituation and trial experiments both
print("Total number of records: " + str(NUM_FILES_ALL))

'''
Each participant took part in 7 experiments, performing at most one experiment per day. 
The first 2 experiments were used for habituation, and the last 5 trials were used for collecting the data. 
The first 7 experiments belong to Subject 1, the next 7 experiments belong to Subject 2, and go on. 
However, Subject 5 could not complete the last experiment. 
Therefore, there are 34 records in the dataset.
'''


img_folder_path = '/'.join(filenames[0].split('/')[:-1])
filenames_trials = []  # remove records of habituation first, second of each subject's set out of 7
for i in range(3, NUM_FILES_ALL, 7):
    for j in range(i, min(i+5, NUM_FILES_ALL+1)):
        filenames_trials.append(img_folder_path+'/'+'eeg_record'+str(j)+'.mat')

print(filenames_trials)
NUM_FILES = len(filenames_trials)
print("Number of trial experiment records: " + str(NUM_FILES))

Total number of records: 34
['/content/data/EEG_Data/eeg_record3.mat', '/content/data/EEG_Data/eeg_record4.mat', '/content/data/EEG_Data/eeg_record5.mat', '/content/data/EEG_Data/eeg_record6.mat', '/content/data/EEG_Data/eeg_record7.mat', '/content/data/EEG_Data/eeg_record10.mat', '/content/data/EEG_Data/eeg_record11.mat', '/content/data/EEG_Data/eeg_record12.mat', '/content/data/EEG_Data/eeg_record13.mat', '/content/data/EEG_Data/eeg_record14.mat', '/content/data/EEG_Data/eeg_record17.mat', '/content/data/EEG_Data/eeg_record18.mat', '/content/data/EEG_Data/eeg_record19.mat', '/content/data/EEG_Data/eeg_record20.mat', '/content/data/EEG_Data/eeg_record21.mat', '/content/data/EEG_Data/eeg_record24.mat', '/content/data/EEG_Data/eeg_record25.mat', '/content/data/EEG_Data/eeg_record26.mat', '/content/data/EEG_Data/eeg_record27.mat', '/content/data/EEG_Data/eeg_record28.mat', '/content/data/EEG_Data/eeg_record31.mat', '/content/data/EEG_Data/eeg_record32.mat', '/content/data/EEG_Data/eeg_re

In [ ]:
'''
Every experiments have same labels:
Focuse: 0 to 10 min
Unfocused: 10 to 20min
Sleep: 20 to end of experiments
'''

train_data = []

for fn in filenames_trials:
    record = loadmat(fn)
    mdata = record['o']  # main object 
    ndata = {n: mdata[n][0,0] for n in mdata.dtype.names}
    
    nS = ndata['nS'][0][0] # variable number of samples
    marker = ndata['marker']
    timestamp = ndata['timestamp']
    data = ndata['data']
    trials = ndata['trials']

    print('marker:', marker.shape)
    print('timestamp:', timestamp.shape)
    print('data:', data.shape)
    print('trials:', trials.shape)
    print()

    train_data.append((nS, marker, data, trials))


print(len(train_data))

In [ ]:
'''
Each Matlab file contains the object of the data acquired from EMOTIV device during one experiment. The raw data is contained in o.data, which is array of size {number-of-samples}x25, thus o.data(:,i) comprises one data channel. The sampling frequency is 128 Hz. The list of data channels and their numerical ids is given below per EMOTIV documentation;

1-'EDCOUNTER' 2-'EDINTERPOLATED'
3-'EDRAWCQ'
4-'EDAF3' 5-'EDF7'
6-'EDF3' 7-'EDFC5'
8-'EDT7' 9-'EDP7'
10-'EDO1' 11-'EDO2'
12-'EDP8' 13-'EDT8'
14-'EDFC6' 15-'EDF4'
16-'EDF8' 17-'EDAF4'
18-'EDGYROX' 19-'EDGYROY'
20-'EDTIMESTAMP' 21-'EDESTIMESTAMP' 22-'EDFUNCID' 23-'EDFUNCVALUE' 24-'EDMARKER'
25'EDSYNCSIGNAL'

The EEG data is in the channels 4:17.
'''